In [1]:
import torch.optim as optim


In [ ]:
#adadleta
for rho in [0, 0.5, 0.75, 0.99]:
    optimizer = optim.Adadelta(net.parameters(), lr=lr, rho=rho)
    state = optimizer.state
    
    tmp_null_weigth = []
    for i in range(len(null_weigth)):
        tmp_null_weigth.append(np.copy(null_weigth[i]))
    net.apply(init_weigths)

    learn_epoch_loss = []
    valid_epoch_loss = []
    test_epoch_loss = []
    epoch_list = []
    
    #для rmsprop
    l1_w11_33 = []
    l2_w7_11 = []
    l3_w1_5 = []
    epoch_mean = []
    epoch_max = []
    epoch_min = []
    
    for epoch in range(550):
        loss_acc = 0
        min_acc = 100
        max_acc = 0
        mean_acc = 0
        for learn_data in learn_dataloader:
            optimizer.zero_grad()
            features_learn, labels_learn = learn_data[:, feature_index], learn_data[:, label_indx]
            features_learn, labels_learn  = autograd.Variable((features_learn).float()).unsqueeze(1), autograd.Variable(labels_learn.float()).unsqueeze(1)
            outputs_learn = net(features_learn)
            loss_learn = criterion(outputs_learn, labels_learn)
            loss_learn.backward()
            optimizer.step()
            loss_acc += float(loss_learn.data)
        learn_epoch_loss.append(loss_acc/len(learn_dataloader))

        if epoch % 10 == 0:
            epoch_list.append(epoch)
            for valid_data in valid_dataloader:
                features_valid, labels_valid = valid_data[:, feature_index], valid_data[:, label_indx]
                features_valid, labels_valid  = autograd.Variable((features_valid).float()).unsqueeze(1), autograd.Variable(labels_valid.float()).unsqueeze(1)
                outputs_valid = net(features_valid)
                loss_valid = criterion(outputs_valid, labels_valid)
                valid_epoch_loss.append(float(loss_valid.data))

            for test_data in test_dataloader:
                features_test, labels_test = test_data[:, feature_index], test_data[:, label_indx]
                features_test, labels_test  = autograd.Variable((features_test).float()).unsqueeze(1), autograd.Variable(labels_test.float()).unsqueeze(1)
                outputs_test = net(features_test)
                loss_test = criterion(outputs_test, labels_test)
                test_epoch_loss.append(float(loss_test.data))
                
        list_of_lr = list(state.values())
        for k in range(len(list_of_lr)):
            square_avg = list_of_lr[k]['square_avg']
            acc_delta = list_of_lr[k]['acc_delta']
            
            std = square_avg.add(1e-7).sqrt_()
            delta = lr * acc_delta.add(1e-7).sqrt_().div_(std)
            
            min_acc = min(min_acc, float(torch.min(avg)))
            max_acc = max(max_acc, float(torch.max(avg)))
            mean_acc += float(torch.mean(avg))
            
        epoch_max.append(max_acc)
        epoch_mean.append(mean_acc/len(list_of_lr))
        epoch_min.append(min_acc)
        
        square_avg = list_of_lr[0]['square_avg']
        avg = lr*square_avg.sqrt().add_(1e-08)
        l1_w11_33.append(float(avg[11, 33]))
        
        square_avg = list_of_lr[2]['square_avg']
        avg = lr*square_avg.sqrt().add_(1e-08)
        l2_w7_11.append(float(avg[7, 11]))
        
        square_avg = list_of_lr[4]['square_avg']
        avg = lr*square_avg.sqrt().add_(1e-08)
        l3_w1_5.append(float(avg[0, 5]))

    plt.clf()
    plt.figure(figsize=(15, 12))
    plt.subplot(4, 3, 7)
    plt.grid(True)
    plt.plot(epoch_max, label='max')
    plt.legend()
    
    plt.subplot(4, 3, 8)
    plt.grid(True)
    plt.plot(epoch_mean, label='mean')
    plt.legend()
    
    plt.subplot(4, 3, 9)
    plt.grid(True)
    plt.plot(epoch_min, label='min')
    plt.legend()
    
    plt.subplot(4, 3, 10)
    plt.grid(True)
    plt.plot(l1_w11_33, label='l1 w1 33')
    plt.legend()
    
    plt.subplot(4, 3, 11)
    plt.grid(True)
    plt.plot(l2_w7_11, label='l2 w7 11')
    plt.legend()
    
    plt.subplot(4, 3, 12)
    plt.grid(True)
    plt.plot(l3_w1_5, label='l3 w1 5')
    plt.legend()
    
    plt.subplot(2, 1, 1)
    plt.grid(True)
    plt.plot(learn_epoch_loss, color='r', label='learn') 
    plt.plot(epoch_list, valid_epoch_loss, color='b', label='valid') 
    plt.plot(epoch_list,  test_epoch_loss, color='g', label='test')
    plt.legend()
    plt.title('alpha = '+str(alpha))
    plt.savefig('rmsprop '+str(alpha)+' learn: '+str(learn_epoch_loss[len(learn_epoch_loss)-1])+' test '+str(test_epoch_loss[len(test_epoch_loss)-1])+'.png')